# Clean up Ohio Ownership Data

The ownership data for Ohio facilities was parsed from the 2016 to 2020 Ohio cost reports we received through a records request. Those reports gave us ownership data and changes as a granular level. The ownership information that was gathered from those cost reports is then combined with the data from QCOR. In addition, Ohio provided additional ownership information about some facilities. That information is also incorporated below.

In [1]:
import pandas as pd

## Ohio Data

In [2]:
RAW_OWNERSHIP_DIR = "../../data/ownership/raw/"
INTERMEDIATE_OWNERSHIP_DIR = "../../data/ownership/intermediate/"

In [3]:
state = pd.read_csv(INTERMEDIATE_OWNERSHIP_DIR + "ohio-ownership-from-cost-reports.csv")

In [4]:
state.head()

,MEDICARE_PROV_NUM,legal_owner,date_acquired,is_mentor,is_brightspring
0,36G447,BrightSpring,NaN,False,True
1,36G261,WERNER HOME,NaN,False,False
2,36G458,ROSE-MARY GRAND HOUSE,NaN,False,False
3,36G440,NCC ROYALTON HOUSE,NaN,False,False
4,36G448,Mentor,01/01/2020,True,False


In [5]:
len(state)

486

## QCOR in Ohio

In [6]:
oh_facs = (
    pd
    .read_csv("../../data/qcor/facilities.csv")
    .loc[
        lambda x: (x["state"] == "OH") & 
        (x["termination_date"].isnull() | (x["termination_date"] >= "2017-03-01"))
    ]
)

In [7]:
len(oh_facs)

487

In [8]:
oh_facs.head()

,name,provider_id,type,region,state,address,phone,particip_date,certified_beds,hospital_based,ownership_type,termination_code,termination_date
4316,MANOR HOME,36G005,Intermediate Care Facilities for Individuals w...,(V) Chicago,OH,"246 N BROADWAY\nGENEVA, OH 44041",440 466-1808,1982-10-01,47,No,For Profit,NaN,NaN
4318,BOYD'S KINSMAN HOME,36G015,Intermediate Care Facilities for Individuals w...,(V) Chicago,OH,"7929 STATE ROUTE 5\nKINSMAN, OH 44428",330 876-5581,1976-08-21,40,No,Government,NaN,NaN
4319,HATTIE LARLHAM CENTER FOR CHILDREN WITH DISABI...,36G017,Intermediate Care Facilities for Individuals w...,(V) Chicago,OH,"9772 DIAGONAL RD\nMANTUA, OH 44255",330 274-2272,1976-05-01,124,No,Non-Profit,NaN,NaN
4320,STEWART LODGE,36G019,Intermediate Care Facilities for Individuals w...,(V) Chicago,OH,"7774 WARNER RD\nMADISON, OH 44057",440 428-7121,1975-03-01,49,No,For Profit,NaN,NaN
4321,LUTHER HOME OF MERCY,36G023,Intermediate Care Facilities for Individuals w...,(V) Chicago,OH,"5810 N MAIN ST\nWILLISTON, OH 43468",419 836-7741,1975-03-01,97,No,Non-Profit,NaN,NaN


## Additional Ohio Information

BuzzFeed News received additional ownership information from Ohio through a public records request. That information is incorporated below.

In [9]:
state_extra = pd.read_excel(RAW_OWNERSHIP_DIR + "2021-11-08-ohio-ICF Licensee with Medicare ID -Public Request_.xlsx")

In [10]:
extra_state_facs = (
    state_extra
    .loc[
        lambda x: ~x["ICF Medicare ID"].isin(state["MEDICARE_PROV_NUM"])
    ]
    .rename(columns={"Agency Name": "legal_owner", "ICF Medicare ID": "MEDICARE_PROV_NUM"})
    .assign(is_brightspring = lambda df: df["legal_owner"] == "ResCare Inc") # ResCare is a subsidiary of BrightSpring
    .assign(date_acquired = None)
    [["MEDICARE_PROV_NUM", "legal_owner", "is_brightspring", "date_acquired"]]
)

In [11]:
total_state = pd.concat([ state, extra_state_facs ])

There are four facilities without a legal owner. Three were opened in 2017, while the fourth was opened more recently. Additional research by BuzzFeed News found that 460 ABBE ROAD does belong to BrightSpring, and we will mark it as such below.

In [12]:
len(oh_facs.loc[lambda x: ~x["provider_id"].isin(total_state["MEDICARE_PROV_NUM"])])

4

In [13]:
(
    oh_facs
    .loc[
        lambda x: ~x["provider_id"]
        .isin(
            total_state["MEDICARE_PROV_NUM"]
        )
    ][[
        "name", 
        "provider_id", 
        "address", 
        "particip_date", 
        "termination_date"
    ]]
)

,name,provider_id,address,particip_date,termination_date
4517,CHENITA GROUP HOME,36G385,"245 WEST FOURTH STREET\nMANSFIELD, OH 44903",1990-01-25,2017-03-03
4688,YOUNGSTOWN DEVELOPMENTAL CENTER,36G589,"4891 EAST MINERAL RIDGE ROAD\nMINERAL RIDGE, O...",2003-05-30,2017-06-30
4693,MONTGOMERY DEVELOPMENTAL CENTER,36G594,"7650 TIMBERCREST DRIVE\nHUBER HEIGHTS, OH 45424",2003-07-28,2017-06-30
4848,460 ABBE ROAD,36G756,"460 ABBE ROAD\nSHEFFIELD LAKE, OH 44054",2021-10-25,NaN


In [14]:
ohio_merged = (
    pd
    .merge(
        oh_facs,
        total_state,
        how="left",
        left_on="provider_id",
        right_on="MEDICARE_PROV_NUM"
    )
)

In [15]:
def fill_is_brightspring(row):
    if pd.isnull(row["is_brightspring"]):
        if row["name"] == "460 ABBE ROAD": # This recently opened facility is owned by ResCare
            return True
        else:
            return False
    else:
        return row["is_brightspring"]

In [16]:
ohio_merged["is_brightspring"] = ohio_merged.apply(lambda x: fill_is_brightspring(x), axis=1)

In [17]:
# Just checking it worked
ohio_merged.loc[
    lambda x: x["name"] == "460 ABBE ROAD"
][["name", "is_brightspring"]]

,name,is_brightspring
481,460 ABBE ROAD,True


In [18]:
ohio_merged[[
    "name", "provider_id", "type", "region", "state", "address",
    "phone", "address", "particip_date", "certified_beds",
    "hospital_based", "ownership_type", "termination_code",
    "termination_date", "legal_owner", "is_brightspring"
]].to_csv("../../data/ownership/final/oh.csv", index=None)

---

---

---